# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298478253605                   -0.85    5.1
  2   -8.300204207848       -2.76       -1.25    1.1
  3   -8.300444182960       -3.62       -1.89    3.0
  4   -8.300461806089       -4.75       -2.77    2.2
  5   -8.300464479027       -5.57       -3.13    3.4
  6   -8.300464567427       -7.05       -3.29    1.2
  7   -8.300464607343       -7.40       -3.43    1.4
  8   -8.300464628587       -7.67       -3.57    1.0
  9   -8.300464639679       -7.95       -3.74    1.0
 10   -8.300464642834       -8.50       -3.92    1.0
 11   -8.300464643849       -8.99       -4.26    1.0


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.63688645903                   -0.71    5.6
  2   -16.67758805915       -1.39       -1.14    1.4
  3   -16.67922379368       -2.79       -1.87    4.6
  4   -16.67928059766       -4.25       -2.60    4.4
  5   -16.67928614320       -5.26       -3.13    4.0
  6   -16.67928621695       -7.13       -3.51    3.1
  7   -16.67928622324       -8.20       -4.20    1.0


In [5]:
self_consistent_field(aluminium_setup(4); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32686292675                   -0.56    7.0
  2   -33.33436241241       -2.12       -1.00    1.1
  3   -33.33596989179       -2.79       -1.72    7.9
  4   -33.33678590690       -3.09       -2.23    5.0
  5   -33.33691955058       -3.87       -2.63    8.5
  6   -33.33694385083       -4.61       -3.76    2.2
  7   -33.33694387812       -7.56       -3.94    4.9
  8   -33.33694392455       -7.33       -4.63    1.8


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298328799789                   -0.85    5.1
  2   -8.300255704377       -2.72       -1.59    1.0
  3   -8.300421893021       -3.78       -2.39    5.0
  4   -8.300310352327   +   -3.95       -2.16    7.2
  5   -8.300464582741       -3.81       -3.82    2.6
  6   -8.300464590558       -8.11       -3.91    3.9
  7   -8.300464631513       -7.39       -4.22    2.0


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32692297932                   -0.56    8.1
  2   -33.33168851787       -2.32       -1.28    1.1
  3   -28.23182028357   +    0.71       -0.71    7.5
  4   -33.17394990793        0.69       -1.45    5.5
  5   -33.30664900609       -0.88       -1.51    4.0
  6   -33.28334653683   +   -1.63       -1.64    3.6
  7   -32.91790288449   +   -0.44       -1.25    4.9
  8   -33.33454856420       -0.38       -2.28    4.2
  9   -33.33455410782       -5.26       -2.32    2.4
 10   -33.33496236832       -3.39       -2.36    1.4
 11   -33.33561921817       -3.18       -2.46    1.0
 12   -33.33679145211       -2.93       -2.90    2.5
 13   -33.33688301731       -4.04       -3.09    3.4
 14   -33.33691095793       -4.55       -3.21    1.6
 15   -33.33693592225       -4.60       -3.56    3.9
 16   -33.33694388391       -5.10       -4.16    2.4


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.